In [1]:
import os 
import numpy as np
import random
import math
from sklearn.metrics import average_precision_score
import pandas as pd

In [2]:
#從grid_search_weight中找尋不同的file 
path = '../Data/grid_search_weight/0212/'
files = os.listdir(path)
len(files)
np.load(path+files[0])['U'].shape

(1489, 64)

In [3]:
def square_sum(pars):
    all_squ_sum = []
    for par in pars:
        all_squ_sum.append(np.sum(np.multiply(par,par)))
    return all_squ_sum 
def shape_sum(pars):
    all_shape = []
    for par in pars:
        all_shape.append(par.shape)
    return all_shape
def relu(x):
    return np.maximum(0,x)  

def softmax(x):
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x 

def sigmoid(z):
    return 1 / (1 + np.exp(-z))  

In [12]:
user_following = np.load('../Data/npy/user_following_1489.npy')
all_3374 = np.load('../Data/npy/all_3239_img0.5.npy')
user_category = np.load('../Data/npy/user_category_1489.npy')
YouTuber_category = np.load('../Data/npy/YouTuber_category_0.7.npy')
active_users = np.load('../Data/npy/active_userID_1489.npy')
all_3374.shape

(88, 3239)

In [13]:
#over5 = 0
#for num in YouTuber_followers:
#    if num >= 5:
#        over5+=1
#print('The num of followers over 5:',over5)
user_category_norm = np.zeros(user_category.shape)
for i in range(len(user_category)):
    user_category_norm[i] = user_category[i]/np.max(user_category[i])
print('user_category after normalized by max...')
print('user_category_norm shape ',user_category_norm.shape)
following_true = [0]*len(user_following)
for i in range(len(user_following)):
    each_user = []
    for j in range(len(user_following[i])):
        if user_following[i][j] == 1:
            each_user.append(j)
    following_true[i] = each_user
#print(following_true)
#最少跟最多的following 
minlen = 10000
maxlen = 0
num_of_follower = []
for i in range(len(following_true)):
    if len(following_true[i]) < minlen:
        minlen = len(following_true[i])
    if len(following_true[i]) > maxlen:
        maxlen = len(following_true[i])
    num_of_follower.append(len(following_true[i]))
print('Min number of followings ',minlen)
print('Max number of followings ',maxlen)
test_amount = 150
yt_test_amount = 18
user_idx = [i for i in range(len(user_following))]
#user_idx = user_idx_over10
#test_idx is the number of user for testing
random.seed(5)
test_idx = random.sample(user_idx,test_amount)
# Training  and Testing --New
train_t = [0]*(len(user_following))
train_f = [0]*(len(user_following))
# Testing 
test_t = [0]*test_amount
test_f = [0]*test_amount
test_pos = -1

for i in range(len(user_following)):
    t_for_train = []
    f_for_train = []
    if i not in test_idx: #if not in test id, just append it to true or false list
        for j in range(88):
            if user_following[i][j] == 1:
                t_for_train.append(j)
            else:
                f_for_train.append(j)
        train_t[i] = t_for_train
        train_f[i] = f_for_train
        
    else: #if in test id, choose 2 true and other 
        test_pos += 1
        temp_t = []
        temp_f = []
        for j in range(88):
            if user_following[i][j] == 1:
                temp_t.append(j)
            else:
                temp_f.append(j)
        #print(len(temp_t),math.ceil(0.5*len(temp_t)))
        t_for_test = random.sample(temp_t,math.ceil(0.5*len(temp_t)))
        f_for_test  = random.sample(temp_f,yt_test_amount-len(t_for_test))
        
        test_t[test_pos] = t_for_test
        test_f[test_pos] = f_for_test
        
        #other for training
        t_for_train = [item for item in temp_t if not item in t_for_test]
        #print(len(t_for_train ))
        f_for_train = [item for item in temp_f if not item in f_for_test]
        train_t[i] = t_for_train
        train_f[i] = f_for_train
total_test = 0
for t in test_t:
    total_test += len(t)
avg = total_test/test_amount
print(avg)

user_category after normalized by max...
user_category_norm shape  (1489, 17)
Min number of followings  5
Max number of followings  34
5.466666666666667


In [14]:
def getScoreMatrix(RS):
    #取出test的資料
    testRS = np.zeros((test_amount,yt_test_amount)) #shape 150*20
    target = np.zeros((test_amount,yt_test_amount))
    #test_t 是true的
    #test_f 是false的

    for z in range(test_amount):
        user_id = test_idx[z]
        #positive target YouTuber list
        youtube_t = test_t[z] 
        #not target YouTuber list
        youtube_f = test_f[z]

        #前兩個放target的RS
        for i in range(len(youtube_t)):
            testRS[z][i] = RS[z][youtube_t[i]]
            target[z][i] = 1
        for i in range(len(youtube_f)):
            testRS[z][i+len(youtube_t)] = RS[z][youtube_f[i]]
    sumtarget = 0
    for i in range(len(target)):
        #print(np.sum(target[i]))
        sumtarget += np.sum(target[i])
    print('num of positive data in testing:',sumtarget)
    print('total testing data:',test_amount*yt_test_amount)
    return target, testRS,sumtarget

def topN(sortlist,n):
    topList = []
    for i in range(n):
        topList.append(sortlist.index(max(sortlist)))
        #print(max(sortlist))
        #print(sortlist.index(max(sortlist)))
        sortlist[sortlist.index(max(sortlist))] = -1000000000
    return topList

def F1_score(prec,rec):
    f1 = (2*prec*rec)/(prec+rec)
    return f1

In [15]:
def getTOP1(target,testRS,sumtarget):
    print('top1')
    correct = 0
    for i in range(len(testRS)):
        top_0 = topN(list(testRS[i]),1) #取一個
        
        #print(top_0)
        if top_0[0] < int(np.sum(target[i])):
            correct += 1
    top1_prec = correct/len(testRS)
    top1_recall = correct/(sumtarget)
    print('prec ',top1_prec,'recall ',top1_recall)
    #f1 score
    print('F1_score:',F1_score(top1_prec,top1_recall))
    return top1_prec,top1_recall,F1_score(top1_prec,top1_recall)
def getTOP3(target,testRS,sumtarget):
    print('top3')
    correct = 0
    for i in range(len(testRS)):
        top_3 = topN(list(testRS[i]),3) #取一個
        
        #print(top_3)
        for j in range(len(top_3)):
            if top_3[j] < int(np.sum(target[i])):
                correct += 1
    top3_prec = correct/(len(testRS)*3)
    top3_recall = correct/(sumtarget)
    print('prec ',top3_prec,'recall ',top3_recall)
    #f1 score
    print('F1_score:',F1_score(top3_prec,top3_recall))
    return top3_prec,top3_recall,F1_score(top3_prec,top3_recall)
def getTOP5(target,testRS,sumtarget):
    print('top5')
    correct = 0
    for i in range(len(testRS)):
        top_5 = topN(list(testRS[i]),5) #取一個
       
        #print(top_5)
        for j in range(len(top_5)):
            if top_5[j] < int(np.sum(target[i])):
                correct += 1
    top5_prec = correct/(len(testRS)*5)
    top5_recall = correct/(sumtarget)
    print('prec ',top5_prec,'recall ',top5_recall)
    #f1 score
    print('F1_score:',F1_score(top5_prec,top5_recall))
    return top5_prec,top5_recall,F1_score(top5_prec,top5_recall)

In [16]:
"""
NDCG
https://daiwk.github.io/posts/nlp-ndcg.html
"""
# pre_list
"""
test_amount = 150
yt_test_amount = 18
"""
def NDCG(target,testRS):
    all_sort = []
    num_ndcg = 10
    pre_matrix = np.zeros(shape=(test_amount,yt_test_amount))
    for i in range(test_amount):
        top_n = topN(list(testRS[i]),num_ndcg) #取一個
        #print(top_n)
        all_sort.append(topN(list(testRS[i]),len(testRS[i])))
        for j in range(len(top_n)):
            pre_matrix[i][top_n[j]] = 1

    #Ideal DCG，理想状况下的DCG。也就是说，相关性完全由高到低排序时算出的DCG：
    def IDCG(ideal_list): #ideal_list example = [1,1,1,1,1,0,0,....]
        idcg=0
        #print('ideal',ideal_list)
        for i in range(len(ideal_list)):
            #print((2**true_list[i]-1),math.log2(i+2))
            idcg+= (2**ideal_list[i]-1)/ math.log2(i+2)
        #print('idcg',idcg)
        return idcg
    def DCG(prec_list): #找出前n名的[1,1,1,0,...]
        dcg=0
        print('prec',prec_list)
        for i in range(len(prec_list)):
            dcg+= (2**prec_list[i]-1)/ math.log2(i+2)
        #print('dcg',dcg)
        return dcg
    total_ndcg = 0
    
    for m in range(test_amount): # the number of testing users
        idcg = IDCG(target[m][:num_ndcg])
        pre_list = []
        least_pre_list = []
        for s in all_sort[m][:num_ndcg]:
            #print(s)
            #print(target[m][s])
            pre_list.append(target[m][s]) #把prec_list 的 score加進去
        for s in all_sort[m][num_ndcg:]:
            #print(s)
            #print(target[m][s])
            least_pre_list.append(target[m][s]) #把prec_list 的 score加進去
        dcg = DCG(pre_list)
        ndcg = dcg/idcg
        #print(ndcg)
        total_ndcg += ndcg
    avg_ndcg = total_ndcg/test_amount
    print('NDCG:',avg_ndcg)
    return pre_matrix,avg_ndcg

# MAP
"""
>>> y_true = np.array([0, 0, 1, 1])
>>> y_scores = np.array([0.1, 0.4, 0.35, 0.8])
>>> average_precision_score(y_true, y_scores)
"""
def MAP(target,pre_matrix):
    print('target:',target)
    print('predict:',pre_matrix)
    total_prec = 0
    for u in range(test_amount):
        y_true = target[u]
        y_scores = pre_matrix[u]
        total_prec+=average_precision_score(y_true, y_scores)
    Map_value = total_prec/test_amount
    print('MAP',Map_value)
    return Map_value

In [17]:
def testing(U, Y, A, E,Au, Ay, Aa, Av,B):
    """
    print('U shape',U.shape)
    print('Y shape',Y.shape)
    print('A shape',A.shape)
    print('E shape',E.shape)
    print('Wv shape',Av.shape)
    print(np.max(Au))
    print(np.max(Ay))
    print(np.max(Aa))
    print(np.max(Av))
    print(np.max(B))
    """
    test_amount = 150
    yt_test_amount = 18
    result=np.zeros((test_amount,88))
    RS=np.zeros((test_amount,88))
    #test_idx --> Test 的 index
    sum_alpha = 0
    for s in range(test_amount):
        #print(s,test_idx[s])

        yes=[]
        #sample=random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]])) #從training part 的positive feedback 取出YouTuber 當成Auxilary
        sample = train_t[test_idx[s]]
        #sample=result_yes_id[now]
        alpha=np.zeros([len(sample)])

        for a in range(len(sample)):
            r =np.max(YouTuber_category[sample[a]]*user_category_norm[test_idx[s]]) #sample a 的category vec *user_category vec
            alpha_a = np.dot(Au[test_idx[s]][sample[a]],np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[test_idx[s]][sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa[test_idx[s]][sample[a]],
                    np.expand_dims(A[sample[a]],0).T)+ np.dot(Av[test_idx[s]][sample[a]],np.dot(E,np.expand_dims(all_3374[sample[a]],0).T))
            """
            relu part ...
            """
            alpha[a]=np.sum((relu(alpha_a)))*r
            """
            tanh part ...
            """
            #alpha[a]=np.sum((np.tanh(alpha_a)))*r
            
            
        mul=np.zeros((1,64))
        #print('alpha--------',alpha)
        #print('add alpha------------',np.add(alpha,0.000000001))
        added_alpha = np.add(alpha,0.0000000001)
        norm_alpha = added_alpha/np.sum(added_alpha)
        #print('alpha-----------',alpha)
        #print('norm alpha--------------',norm_alpha)
        sum_alpha += np.sum(alpha)
        for i in range(len(sample)):
            mul+=norm_alpha[i]*A[sample[i]] #attention alpha*Ai part
        new_mul=mul+U[test_idx[s]]  #(U+auxilary)
    
        
        
        for k in range(88):
            result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
            RS[s][k] = np.dot(new_mul,Y[k].T)+np.dot(B[test_idx[s]],np.dot(E, all_3374[k].T))
            """
            print('================================')
            print('U',np.sum(U[test_idx[s]] ))
            print('A',np.sum(mul))
            print('Y',np.sum(Y[k]))
            print('B',np.sum(B[test_idx[s]]))
            print('E',np.sum(np.dot(E, all_3374[k].T)))
            print('User',np.dot(U[test_idx[s]],Y[k].T))
            print('Aux',np.dot(mul,Y[k].T))
            print('Former',np.dot(new_mul,Y[k].T))
            print('Later',np.dot(B[test_idx[s]],np.dot(E, all_3374[k].T)))
            print('RS',np.dot(new_mul,Y[k].T)+np.dot(B[test_idx[s]],np.dot(E, all_3374[k].T)))
            print('--------------------------------')
            """
            
    print('sum_alpha',sum_alpha)
    return RS

In [18]:
#With Embedding
import os 
#從grid_search_weight中找尋不同的file 
path = '../Data/grid_search_weight/0123_social_15D/'
all_files = os.listdir(path)
print(all_files)

['0.01_0.01_0.001_0.01_Edims100.npz', '0.01_0.01_0.001_0.01_Edims150.npz', '0.01_0.01_0.001_0.01_Edims160.npz', '0.01_0.01_0.001_0.01_Edims170.npz', '0.01_0.01_0.001_0.01_Edims180.npz', '0.01_0.01_0.001_0.01_Edims190.npz', '0.01_0.01_0.001_0.01_Edims200.npz', '0.01_0.01_0.001_0.01_Edims210.npz', '0.01_0.01_0.001_0.01_Edims220.npz', '0.01_0.01_0.001_0.01_Edims230.npz', '0.01_0.01_0.001_0.01_Edims240.npz', '0.01_0.01_0.001_0.01_Edims250.npz', '0.01_0.01_0.001_0.01_Edims300.npz', '0.01_0.01_0.001_0.01_Edims5.npz', '0.01_0.01_0.001_0.01_Edims50.npz']


In [19]:
par_files = list(set([file.split('_Edims')[0] for file in all_files]))
print(par_files)
for par_file in par_files:
    NDCG_List = []
    MAP_List = []
    DIM_List = []
    TOP1_Prec = []
    TOP3_Prec = []
    TOP5_Prec = []
    TOP1_F1 = []
    TOP3_F1 = []
    TOP5_F1 = []
    TOP1_Recall = []
    TOP3_Recall = []
    TOP5_Recall = []
    
    csv_path = '../Data/grid_search_weight/CSV_Tan/'
    for file in all_files:
        if par_file in file:
            print(file)
            par_data = np.load(path+file)
            U = par_data['U']
            Y = par_data['Y']
            A = par_data['A']
            E = par_data['E']
            #W1 = par_data['W1']
            Wu = par_data['Wu']
            Wy = par_data['Wy']
            Wa = par_data['Wa']
            Wv = par_data['Wv']
            B = par_data['B']
            RS = testing(U, Y, A, E,Wu, Wy, Wa, Wv,B)
            target,testRS,sumtarget = getScoreMatrix(RS)
            prec_1,recall_1,f1_1 = getTOP1(target,testRS,sumtarget)
            prec_3,recall_3,f1_3 = getTOP3(target,testRS,sumtarget)
            prec_5,recall_5,f1_5 = getTOP5(target,testRS,sumtarget)
            pre_matrix,avg_ndcg = NDCG(target,testRS)
            Map_value = MAP(target,pre_matrix)
            NDCG_List.append(avg_ndcg)
            MAP_List.append(Map_value)
            DIM_List.append(file.split('Edims')[1].split('.npz')[0])
            TOP1_Prec.append(prec_1)
            TOP1_Recall.append(recall_1)
            TOP1_F1.append(f1_1)
            TOP3_Prec.append(prec_3)
            TOP3_Recall.append(recall_3)
            TOP3_F1.append(f1_3)
            TOP5_Prec.append(prec_5)
            TOP5_Recall.append(recall_5)
            TOP5_F1.append(f1_5)
            print('--------------------------------------------------------------------------------------------')
    #print(NDCG_List)
    #print(DIM_List)
    result_dict = {'Dims':DIM_List,'NDCG':NDCG_List,'MAP':MAP_List,
                   'TOP1 Precision':TOP1_Prec,'TOP1 Recall':TOP1_Recall,'TOP1 F1':TOP1_F1,
                  'TOP3 Precision':TOP3_Prec,'TOP3 Recall':TOP3_Recall,'TOP3 F1':TOP3_F1,
                  'TOP5 Precision':TOP5_Prec,'TOP5 Recall':TOP5_Recall,'TOP5 F1':TOP5_F1}
    df = pd.DataFrame(result_dict)
    df.to_csv(csv_path+par_file+'.csv',index=False)

['0.01_0.01_0.001_0.01']
0.01_0.01_0.001_0.01_Edims100.npz
sum_alpha 0.2847922565719009
num of positive data in testing: 820.0
total testing data: 2700
top1
prec  0.6533333333333333 recall  0.11951219512195121
F1_score: 0.2020618556701031
top3
prec  0.6 recall  0.32926829268292684
F1_score: 0.4251968503937008
top5
prec  0.5533333333333333 recall  0.5060975609756098
F1_score: 0.5286624203821655
prec [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
prec [0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0]
prec [1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
prec [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
prec [0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0]
prec [1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0]
prec [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0]
prec [1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0]
prec [1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]
prec [1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0]
prec [1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.

sum_alpha 0.2535676041717824
num of positive data in testing: 820.0
total testing data: 2700
top1
prec  0.6733333333333333 recall  0.12317073170731707
F1_score: 0.20824742268041238
top3
prec  0.6266666666666667 recall  0.3439024390243902
F1_score: 0.44409448818897634
top5
prec  0.5893333333333334 recall  0.5390243902439025
F1_score: 0.5630573248407644
prec [1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
prec [1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0]
prec [1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]
prec [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
prec [0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0]
prec [1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0]
prec [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0]
prec [0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0]
prec [1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
prec [0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0]
prec [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0]
prec [1.0, 1.0, 0.0, 1.0, 1.0,

sum_alpha 0.23733459388082181
num of positive data in testing: 820.0
total testing data: 2700
top1
prec  0.6 recall  0.10975609756097561
F1_score: 0.18556701030927836
top3
prec  0.5644444444444444 recall  0.3097560975609756
F1_score: 0.4
top5
prec  0.5493333333333333 recall  0.5024390243902439
F1_score: 0.5248407643312103
prec [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
prec [1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
prec [1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0]
prec [0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0]
prec [0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0]
prec [1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0]
prec [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0]
prec [0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0]
prec [1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
prec [1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0]
prec [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0]
prec [1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0]
prec

sum_alpha 0.21474026407317004
num of positive data in testing: 820.0
total testing data: 2700
top1
prec  0.6866666666666666 recall  0.12560975609756098
F1_score: 0.21237113402061858
top3
prec  0.6333333333333333 recall  0.3475609756097561
F1_score: 0.44881889763779526
top5
prec  0.6066666666666667 recall  0.5548780487804879
F1_score: 0.5796178343949044
prec [1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
prec [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
prec [1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
prec [1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0]
prec [0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0]
prec [1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0]
prec [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0]
prec [0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0]
prec [1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]
prec [1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0]
prec [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0]
prec [1.0, 1.0, 0.0, 1.0, 1.0

sum_alpha 0.32306814942438383
num of positive data in testing: 820.0
total testing data: 2700
top1
prec  0.66 recall  0.12073170731707317
F1_score: 0.20412371134020618
top3
prec  0.6355555555555555 recall  0.348780487804878
F1_score: 0.45039370078740154
top5
prec  0.5853333333333334 recall  0.5353658536585366
F1_score: 0.5592356687898089
prec [1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
prec [1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]
prec [1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0]
prec [0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0]
prec [1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0]
prec [1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0]
prec [1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]
prec [0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0]
prec [1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
prec [1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0]
prec [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0]
prec [1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0

...

sum_alpha 0.2724349110644678
num of positive data in testing: 820.0
total testing data: 2700
top1
prec  0.4866666666666667 recall  0.08902439024390243
F1_score: 0.15051546391752577
top3
prec  0.5088888888888888 recall  0.27926829268292686
F1_score: 0.3606299212598425
top5
prec  0.48533333333333334 recall  0.44390243902439025
F1_score: 0.4636942675159236
prec [1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
prec [1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
prec [1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0]
prec [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]
prec [0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0]
prec [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
prec [0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0]
prec [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0]
prec [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0]
prec [0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]
prec [0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0]
prec [1.0, 1.0, 0.0, 1.0, 0.

sum_alpha 0.31510775454895057
num of positive data in testing: 820.0
total testing data: 2700
top1
prec  0.5733333333333334 recall  0.1048780487804878
F1_score: 0.177319587628866
top3
prec  0.6022222222222222 recall  0.3304878048780488
F1_score: 0.42677165354330704
top5
prec  0.5853333333333334 recall  0.5353658536585366
F1_score: 0.5592356687898089
prec [1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
prec [1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]
prec [1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0]
prec [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
prec [1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0]
prec [1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0]
prec [1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
prec [1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0]
prec [0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]
prec [1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0]
prec [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0]
prec [1.0, 1.0, 0.0, 1.0, 1.0, 0

sum_alpha 0.23867535295633
num of positive data in testing: 820.0
total testing data: 2700
top1
prec  0.68 recall  0.12439024390243902
F1_score: 0.21030927835051547
top3
prec  0.6377777777777778 recall  0.35
F1_score: 0.4519685039370079
top5
prec  0.604 recall  0.552439024390244
F1_score: 0.5770700636942676
prec [1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
prec [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
prec [1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0]
prec [1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
prec [1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]
prec [1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0]
prec [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0]
prec [1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0]
prec [1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0]
prec [0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
prec [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0]
prec [1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0]
prec [1.0, 1.0, 1.0

sum_alpha 0.21900395925926258
num of positive data in testing: 820.0
total testing data: 2700
top1
prec  0.66 recall  0.12073170731707317
F1_score: 0.20412371134020618
top3
prec  0.5822222222222222 recall  0.3195121951219512
F1_score: 0.41259842519685036
top5
prec  0.5546666666666666 recall  0.5073170731707317
F1_score: 0.5299363057324841
prec [1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
prec [1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
prec [1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0]
prec [0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0]
prec [0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0]
prec [1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0]
prec [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0]
prec [0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0]
prec [1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0]
prec [1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0]
prec [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0]
prec [1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.

sum_alpha 0.24014795960231658
num of positive data in testing: 820.0
total testing data: 2700
top1
prec  0.66 recall  0.12073170731707317
F1_score: 0.20412371134020618
top3
prec  0.6044444444444445 recall  0.33170731707317075
F1_score: 0.42834645669291344
top5
prec  0.5813333333333334 recall  0.5317073170731708
F1_score: 0.5554140127388536
prec [1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
prec [0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0]
prec [1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]
prec [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
prec [0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0]
prec [0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0]
prec [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0]
prec [1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0]
prec [1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
prec [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0]
prec [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0]
prec [1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1

In [32]:
#With Embedding
import os 
#從grid_search_weight中找尋不同的file 
path = '../Data/grid_search_weight/0113/'
all_files = os.listdir(path)
#print(files)
par_files = list(set([file.split('_Edims')[0] for file in all_files]))
print(par_files)
for par_file in par_files:
    NDCG_List = []
    MAP_List = []
    DIM_List = []
    csv_path = '../Data/grid_search_weight/CSV_Tan/'
    for file in all_files:
        if par_file in file:
            print(par_file,file)
            par_data = np.load(path+file)
            U = par_data['U']
            Y = par_data['Y']
            A = par_data['A']
            E = par_data['E']
            #W1 = par_data['W1']
            Wu = par_data['Wu']
            Wy = par_data['Wy']
            Wa = par_data['Wa']
            Wv = par_data['Wv']
            Wve = par_data['Wve']
            B = par_data['B']
            RS = testing(U, Y, A, E,Wu, Wy, Wa, Wv,Wve,B)
            target,testRS,sumtarget = getScoreMatrix(RS)
            getTOP1(target,testRS,sumtarget)
            getTOP3(target,testRS,sumtarget)
            getTOP5(target,testRS,sumtarget)
            pre_matrix,avg_ndcg = NDCG(target,testRS)
            Map_value = MAP(target,pre_matrix)
            NDCG_List.append(avg_ndcg)
            MAP_List.append(Map_value)
            DIM_List.append(file.split('Edims')[1].split('.npz')[0])
            print('--------------------------------------------------------------------------------------------')
    #print(NDCG_List)
    #print(DIM_List)
    result_dict = {'Dims':DIM_List,'NDCG':NDCG_List,'MAP':MAP_List}
    df = pd.DataFrame(result_dict)
    #df.to_csv(csv_path+par_file+'.csv',index=False)

['0.01_0.01_0.001_0.01']
0.01_0.01_0.001_0.01 0.01_0.01_0.001_0.01_Edims100.npz


KeyError: 'Wve is not a file in the archive'

In [94]:
for i in range(len(NDCG_List)):
    print(NDCG_List[i],MAP_List[i])

# NDCG For Different Dimensions

In [111]:
######## code here !!!!

(10, 20)

In [69]:
def topN(sortlist,n):
    topList = []
    for i in range(n):
        topList.append(sortlist.index(max(sortlist)))
        #print(max(sortlist))
        #print(sortlist.index(max(sortlist)))
        sortlist[sortlist.index(max(sortlist))] = -1000000000
    return topList

In [70]:
count_0_all = []
for i in range(len(testRS)):
    top_0 = topN(list(testRS[i]),int(np.sum(target[i])))
    count_0_all.append(top_0)
    print(top_0)

acc_0 = 0
total = 0
for i in range(len(count_0_all)):
    for j in range(len(count_0_all[i])):
        #print(int(np.sum(target[i])))
        total+=int(np.sum(target[i]))
        if count_0_all[i][j] < int(np.sum(target[i])): #代表是0或1 (也就是target)
            acc_0 += 1
avg_acc = acc_0/100
print('avg_accuarcy for count_0:',avg_acc)
print(acc_0)

[0, 2, 1]
[3, 6, 2]
[1, 2, 13, 4, 5, 3, 17]
[1, 10, 8, 2, 5, 4]
[4, 1, 12]
[1, 15, 11]
[2, 1, 10, 0]
[3, 2, 5, 7, 4, 18]
[3, 18, 8, 5, 13]
[12, 16, 7, 0, 9]
avg_accuarcy for count_0: 0.24
24


# TOP1,TOP3,TOP5

In [71]:
def F1_score(prec,rec):
    f1 = (2*prec*rec)/(prec+rec)
    return f1

# NDCG 

In [172]:
a_list = [0.02162337, 0.02162659, 0.00240219, 0.00780985, 0.02162318, 0.00780592, 0.00780849]
softmax(a_list)

array([0.14409628, 0.14409674, 0.14135303, 0.14211949, 0.14409625,
       0.14211893, 0.14211929])